In [3]:
%load_ext autoreload 
%autoreload 2

In [ ]:
# import packages
import json
import xarray as xr
import dask
import numpy as np
from dask.distributed import Client, LocalCluster
import argparse

import modules
import bc_module_v2

/home/lorenz-c/miniconda3/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [ ]:
year        = 2022
month       = 9
domain_name = 'khuzestan'

In [13]:
# Get some ressourcers
client, cluster = modules.getCluster('rome', 1, 30)
    
# Do the memory magic...
client.amm.start() 
    
# Write some info about the cluster
print(client.scheduler_info)

<bound method Client.scheduler_info of <Client: 'tcp://172.27.80.110:41459' processes=0 threads=0, memory=0 B>>


/home/lorenz-c/miniconda3/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38415 instead
  warnings.warn(


In [14]:
client.dashboard_link

'http://172.27.80.110:38415/status'

In [7]:
client, cluster = modules.getCluster('ivyshort', 2, 40)

/home/lorenz-c/miniconda3/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46282 instead
  warnings.warn(


In [11]:
client

<Client: 'tcp://172.27.80.110:46121' processes=80 threads=80, memory=112.00 GiB>

In [ ]:
print(client.scheduler_info)

In [ ]:
print(f"Dask Dashboard available at {client.dashboard_link}")

In [ ]:
# Read the domain configuration from the respective JSON
with open('conf/domain_config.json', 'r') as j:
    domain_config = json.loads(j.read())

# Read the global configuration from the respective JSON
with open('conf/global_config.json', 'r') as j:
    global_config = json.loads(j.read())

with open('conf/variable_config.json', 'r') as j:
    variable_config = json.loads(j.read())

In [ ]:
# Select the configuration for the actual domain --> We want to do that with the argument parser..
domain_config = domain_config['khuzestan']

In [ ]:
variable_config = { key:value for key,value in variable_config.items() if key in domain_config['variables']}

In [ ]:
raw_dict, bcsd_dict, ref_hist_dict, mdl_hist_dict= modules.set_filenames(year, month, domain_config, variable_config)

In [ ]:
# Read the dimensions for the output file (current prediction)
coords = modules.get_coords_from_files(list(raw_dict.values())[0])

In [ ]:
global_config = modules.update_global_attributes(global_config, domain_config['bc_params'], coords, 'khuzestan')

In [ ]:
encoding = modules.set_encoding(variable_config, coords)

In [ ]:
# Create an empty NetCDF in which we write the BCSD output
ds = modules.create_4d_netcdf(bcsd_dict, global_config, domain_config, variable_config, coords)

In [ ]:
# Loop over each variable
for variable in variable_config:
    
    ds_obs = xr.open_dataset(ref_hist_dict[variable])
    ds_obs = xr.open_mfdataset(ref_hist_dict[variable], chunks={'time': len(ds_obs.time), 'lat': 50, 'lon': 50}, parallel=True, engine='h5netcdf')
    da_obs = ds_obs[variable].persist()
        
    ds_mdl = modules.preprocess_mdl_hist(mdl_hist_dict[variable], month, variable) # chunks={'time': 215, 'year': 36, 'ens': 25, 'lat': 1, 'lon': 1})
    da_mdl = ds_mdl.persist()
        
    ds_pred = xr.open_dataset(raw_dict[variable])
    ds_pred = xr.open_mfdataset(raw_dict[variable], chunks={'time': len(ds_pred.time), 'ens': len(ds_pred.ens), 'lat': 50, 'lon': 50}, parallel=True, engine='h5netcdf')
    da_pred = ds_pred[variable].persist()
    
    # Nasty workaround so that coordinates match for apply_ufunc
    da_pred = da_pred.assign_coords(lon=ds_pred.lon.values.astype(np.float32), lat=ds_pred.lat.values.astype(np.float32))

    dayofyear_obs = ds_obs['time.dayofyear']
    dayofyear_mdl = ds_mdl['time.dayofyear']
        
    da_temp = xr.DataArray(
        None, 
        dims = ['time', 'lat', 'lon', 'ens'], 
        coords = {
        'time': ('time', coords['time'], {'standard_name': 'time', 'long_name': 'time'}),
        'ens': ('ens', coords['ens'], {'standard_name': 'realization', 'long_name': 'ensemble_member'}),
        'lat': ('lat', coords['lat'], {'standard_name': 'latitude', 'long_name': 'latitude', 'units': 'degrees_east'}),
        'lon': ('lon', coords['lon'], {'standard_name': 'longitude', 'long_name': 'longitude', 'units': 'degrees_north'})
        }
    )


    for timestep in range(0, len(ds_pred.time)):
            
        print(f'Correcting timestep {timestep}...')
    
        
        day = dayofyear_mdl[timestep]
    
        day_range = (np.arange(day - domain_config['bc_window'], day + domain_config['bc_window'] + 1) + 365) % 365 + 1
    
        intersection_day_obs = np.in1d(dayofyear_obs, day_range)
        intersection_day_mdl = np.in1d(dayofyear_mdl, day_range)
    
        da_obs_sub = da_obs.loc[dict(time=intersection_day_obs)]
    
        da_mdl_sub = da_mdl.loc[dict(time=intersection_day_mdl)]
        
        da_mdl_sub = da_mdl_sub.stack(ens_time=("ens", "time"), create_index=True)
        da_mdl_sub = da_mdl_sub.drop('time')
    
    
        da_pred_sub = da_pred.isel(time=timestep)
    
    
        pred_corr_act = xr.apply_ufunc(
            bc_module_v2.bc_module, 
            da_pred_sub, 
            da_obs_sub, 
            da_mdl_sub, 
            kwargs={'bc_params': domain_config['bc_params'], 'precip': variable_config[variable]['isprecip']},
            input_core_dims=[["ens"], ["time"], ['ens_time']], 
            output_core_dims=[["ens"]], 
            vectorize=True, 
            dask="parallelized", 
            output_dtypes=[np.float64]) 
        
        da_temp.loc[dict(time=ds_pred.time.values[timestep])] = pred_corr_act
        
        
    # Change the datatype from "object" to "float64" --> Can we somehow get around this???
    da_temp = da_temp.astype('float64')
        
    # Select only the actual variable from the output dataset
    ds_out_sel = ds[[variable]]
        
    # Fill this variable with some data...
    ds_out_sel[variable].values = da_temp.transpose('time', 'ens', 'lat', 'lon').values
        
    # ...and save everything to disk..
    ds_out_sel.to_netcdf(bcsd_dict[variable], mode='a', format='NETCDF4_CLASSIC', engine='netcdf4', encoding = {variable: encoding[variable]})


In [ ]:
da_temp[0,:,:,0].plot()

In [12]:
client.close()
cluster.close()

In [ ]:
    # Loop over each variable
    for variable in variable_attributes:
     

        ###### Old IO-Module #####
        # load data as dask objects
        # Obs (1981 - 2016 on daily basis)
        ds_obs = xr.open_mfdataset(obs_dict[variable], chunks={'time': 13149, 'lat': 50, 'lon': 50}, parallel=True, engine='h5netcdf')
        ds_obs = ds_obs[variable].persist()
        
        # Mdl (historical, 1981 - 2016 for one month and 215 days)  215, 36, 25, 1, 1 ;
        # Preprocess historical mdl-data, create a new time coord, which contain year and day at once and not separate
        ds_mdl = modules.preprocess_mdl_hist(mdl_dict[variable], month) # chunks={'time': 215, 'year': 36, 'ens': 25, 'lat': 1, 'lon': 1})
        ds_mdl = ds_mdl[variable].persist()
        
        # Pred (current year for one month and 215 days)
        ds_pred = xr.open_mfdataset(pred_dict[variable], chunks={'time': 215, 'ens': 51, 'lat': 50, 'lon': 50}, parallel=True, engine='h5netcdf')
        ds_pred = ds_pred[variable].persist()
        
        # Change data type of latidude and longitude, otherwise apply_u_func does not work
        ds_pred = ds_pred.assign_coords(lon=ds_pred.lon.values.astype(np.float32), lat=ds_pred.lat.values.astype(np.float32))

        # Calculate day of the year from time variable
        dayofyear_obs = ds_obs['time.dayofyear']
        dayofyear_mdl = ds_mdl['time.dayofyear']
        
        da_temp = xr.DataArray(
            None, 
            dims = ['time', 'lat', 'lon', 'ens'], 
            coords = {
                'time': ('time', coords['time'], {'standard_name': 'time', 'long_name': 'time'}),
                'ens': ('ens', coords['ens'], {'standard_name': 'realization', 'long_name': 'ensemble_member'}),
                'lat': ('lat', coords['lat'], {'standard_name': 'latitude', 'long_name': 'latitude', 'units': 'degrees_east'}),
                'lon': ('lon', coords['lon'], {'standard_name': 'longitude', 'long_name': 'longitude', 'units': 'degrees_north'})
            }
        )

        if variable == "tp":
            precip = True
        else:
            precip = False
            
        for timestep in range(0, len(ds_pred.time)):
            
            print(f'Correcting timestep {timestep}...')
    
        
            day = dayofyear_mdl[timestep]
    
            day_range = (np.arange(day - bc_params['window'], day + bc_params['window'] + 1) + 365) % 365 + 1
    
            intersection_day_obs = np.in1d(dayofyear_obs, day_range)
            intersection_day_mdl = np.in1d(dayofyear_mdl, day_range)
    
            ds_obs_sub = ds_obs.loc[dict(time=intersection_day_obs)]
    
            ds_mdl_sub = ds_mdl.loc[dict(time=intersection_day_mdl)]
        
            ds_mdl_sub = ds_mdl_sub.stack(ens_time=("ens", "time"), create_index=True)
            ds_mdl_sub = ds_mdl_sub.drop('time')
    
    
            ds_pred_sub = ds_pred.isel(time=timestep)
    
    
            pred_corr_act = xr.apply_ufunc(
                bc_module, 
                ds_pred_sub, 
                ds_obs_sub, 
                ds_mdl_sub, 
                kwargs={'bc_params': bc_params, 'precip': precip},
                input_core_dims=[["ens"], ["time"], ['ens_time']], 
                output_core_dims=[["ens"]], 
                vectorize=True, 
                dask="parallelized", 
                output_dtypes=[np.float64]) 
        
            da_temp.loc[dict(time=ds_pred.time.values[timestep])] = pred_corr_act
        
    
        ds_out[variable] = da_temp.transpose('time', 'ens', 'lat', 'lon')
        
    # We still need to write the NetCDF...

In [5]:
import setup_domain_func

AttributeError: 'NoneType' object has no attribute 'group'